In [1]:
import pandas as pd
import fiona
import geopandas as gpd
from sqlalchemy import create_engine
import numpy as np
#import dedupe
from dotenv import load_dotenv
import os
import pyArango
import usaddress
from usaddress import tag
from scourgify import normalize_address_record
import re
import json

In [2]:
from pyArango.connection import *
conn = Connection(username="root", password="0505")

In [3]:
load_dotenv()
PG_CONNECT = os.getenv("PG_CONNECT")

In [4]:
# create arangoDB database, or open it if it already exists
try:
    corp_db = conn.createDatabase(name="corp_data")
    # from command line:
    # arangoimport --file '''path to CorpData.csv''' --collection corp_data --create-collection true --type csv --server.database corp_data
    # arangoimport --file '''path to CorpIndividualExport.csv''' --collection corp_individual --create-collection true --type csv --server.database corp_data

except:
    corp_db = conn["corp_data"]

In [5]:
# arangoDB database structure:

# corp_db: arangoDB database
# corp_db["corp_data"]: arangoDB collection
# corp_db["corp_data"][17777999]: arangoDB document (one piece of data, or one corporation)

In [10]:
# extracting the values of the indiv_unique_address collection
indiv_aql = "FOR x IN indiv_unique_address RETURN x"
indiv_query_result = corp_db.AQLQuery(indiv_aql,rawResults=True, batchSize = 1000)
col_indiv = {}
ind_indiv = 0

for indiv in indiv_query_result:
    col_indiv[ind_indiv] = indiv
    ind_indiv += 1

indiv_df = pd.DataFrame.from_dict(data = col_indiv, orient = 'index')
indiv_ar = indiv_df.to_numpy()

In [6]:
# extracting the values of the indiv_unique_address collection
indiv_aql = "FOR x IN indiv_unique_name_address RETURN x"
indiv_query_result = corp_db.AQLQuery(indiv_aql,rawResults=True, batchSize = 1000)
col_indiv = {}
ind_indiv = 0

for indiv in indiv_query_result:
    col_indiv[ind_indiv] = indiv
    ind_indiv += 1

indiv_short_df = pd.DataFrame.from_dict(data = col_indiv, orient = 'index')
indiv_short = indiv_short_df.to_numpy()

In [7]:
print(indiv_ar[0])

['56380105' 'indiv_unique_address/56380105' '_ef6tPHC---' '00i6fl'
 'RAMADAN MOHAMAD ZIAD' 'RAMADAN MOHAMAD'
 '85 NEWTON ST., , NORTHBOROUGH, MA']


In [17]:
print(len(indiv_short))

2092355


In [18]:
def indiv2indiv_same(indiv_ar, indiv_short, batch_size): 
    
    for i in range(2000,2093):
        print(i)
        print(i*batch_size,(i+1)*batch_size)
        batch_ar = indiv_short[i*batch_size:(i+1)*batch_size][:] 
        
        for j in range(0,batch_ar.shape[0]):
            id1 = batch_ar[j][1]
            indivs = np.where((indiv_ar[:,7]==batch_ar[j][7]))[0]
            id2 = indiv_ar[indivs][:,1]
            for k in id2:
                edge_attributes = {"_from":id1, "_to": k, "type": 'indiv/same/address'}
                edge = corp_db["indiv2indiv"].createDocument(edge_attributes)
                edge.save()

indiv2indiv_same(indiv_ar_2, indiv_short, 1000)

2000
2000000 2001000
2001
2001000 2002000
2002
2002000 2003000
2003
2003000 2004000
2004
2004000 2005000
2005
2005000 2006000
2006
2006000 2007000
2007
2007000 2008000
2008
2008000 2009000
2009
2009000 2010000
2010
2010000 2011000
2011
2011000 2012000
2012
2012000 2013000
2013
2013000 2014000
2014
2014000 2015000
2015
2015000 2016000
2016
2016000 2017000
2017
2017000 2018000
2018
2018000 2019000
2019
2019000 2020000
2020
2020000 2021000
2021
2021000 2022000
2022
2022000 2023000
2023
2023000 2024000
2024
2024000 2025000
2025
2025000 2026000
2026
2026000 2027000
2027
2027000 2028000
2028
2028000 2029000
2029
2029000 2030000
2030
2030000 2031000
2031
2031000 2032000
2032
2032000 2033000
2033
2033000 2034000
2034
2034000 2035000
2035
2035000 2036000
2036
2036000 2037000
2037
2037000 2038000
2038
2038000 2039000
2039
2039000 2040000
2040
2040000 2041000
2041
2041000 2042000
2042
2042000 2043000
2043
2043000 2044000
2044
2044000 2045000
2045
2045000 2046000
2046
2046000 2047000
2047
2047000 

In [11]:
indiv_df.loc[:,'NamePlusAddr'] = [' '.join((str(a),str(b))) for a,b in zip(indiv_df['IndividualNameShort'],indiv_df['Addr'])]
indiv_ar_2 = indiv_df.to_numpy()
print(indiv_ar_2[0])

['56380105' 'indiv_unique_address/56380105' '_ef6tPHC---' '00i6fl'
 'RAMADAN MOHAMAD ZIAD' 'RAMADAN MOHAMAD'
 '85 NEWTON ST., , NORTHBOROUGH, MA'
 'RAMADAN MOHAMAD 85 NEWTON ST., , NORTHBOROUGH, MA']


In [13]:
indiv_ar_2 = indiv.to_numpy()

In [17]:
unique_keys, indices = np.unique(indiv_ar_2[:,7], return_index=True)
indiv_unique = indiv_ar_2[np.sort(indices)]
print(len(indiv_unique))

2092355


In [26]:
indiv_unique[0]

array(['56380105', 'indiv_unique_address/56380105', '_ef6tPHC---',
       '00i6fl', 'RAMADAN MOHAMAD ZIAD', 'RAMADAN MOHAMAD',
       '85 NEWTON ST., , NORTHBOROUGH, MA',
       'RAMADAN MOHAMAD 85 NEWTON ST., , NORTHBOROUGH, MA'], dtype=object)

In [27]:
indiv_u = np.delete(indiv_unique,(0,1,2),1)

In [29]:
indiv_df_final = pd.DataFrame(indiv_u,columns = ['DataID','IndividualName','IndividualNameShort','Addr','NamePlusAddr'])

# create json and write it to an existing empty json file
result = indiv_df_final.to_json(orient="records")
parsed = json.loads(result)
indiv_json = json.dumps(parsed, indent=4) 

with open('../indiv_name_addr.json', 'w') as outfile:
    outfile.write(indiv_json)

In [18]:
len(indiv_ar)

2581604

In [22]:
indiv_ar = indiv_ar_2

In [7]:
# [3]: DataID; [5]: NameShort; [6]: BusAddr; [7]: ResAddr

def c_1(ind,ind2):
    if ind[5] == ind2[5]:
        if ind[6]!= ', ,' and ((ind[6] == ind2[6]) or (ind[6] == ind2[7])):
            return True
        elif ind[7]!= ', ,' and ((ind[7] == ind2[7]) or (ind[7] == ind2[6])):
            return True
    return False

In [27]:
c_2 = np.vectorize(c_1)

In [40]:
def indiv2indiv_same(indiv_ar, batch_size): 
    
    for i in range(10,15):
        print(i)
        print(i*batch_size,(i+1)*batch_size)
        batch_ar = indiv_ar[i*batch_size:(i+1)*batch_size][:]
            
        for j in range(0,batch_ar.shape[0]):
            for k in indiv_ar:
                if c_1(k,batch_ar[j]) and k[1]!=batch_ar[j][1]:
                    try:
                        placeholder = corp_db["indiv2indiv"].fetchFirstExample({"_to": k[1], "_from": indiv_ar[i*batch_size+j][1]})[0]
                    except:
                        edge_attributes = {"_from": k[1], "_to": batch_ar[j][1], "type": 'indiv2indiv/same'}
                        edge = corp_db["indiv2indiv"].createDocument(edge_attributes)
                        edge.save()

indiv2indiv_same(indiv_ar,1000)

10
10000 11000


KeyboardInterrupt: 

In [ ]:
np.where(indiv_ar)

In [22]:
batch_ar = indiv_ar[0:10]

In [25]:
it = np.nditer(batch_ar.T,flags=["refs_ok","multi_index"])
for i in it.multi_index:
    print(i,it.multi_index)

12936266 (0, 0)
12936268 (0, 1)
12936269 (0, 2)
12936270 (0, 3)
12936271 (0, 4)
12936272 (0, 5)
12936274 (0, 6)
12936275 (0, 7)
12936277 (0, 8)
12936278 (0, 9)
indiv_data_deduped/12936266 (1, 0)
indiv_data_deduped/12936268 (1, 1)
indiv_data_deduped/12936269 (1, 2)
indiv_data_deduped/12936270 (1, 3)
indiv_data_deduped/12936271 (1, 4)
indiv_data_deduped/12936272 (1, 5)
indiv_data_deduped/12936274 (1, 6)
indiv_data_deduped/12936275 (1, 7)
indiv_data_deduped/12936277 (1, 8)
indiv_data_deduped/12936278 (1, 9)
_edj3zGm--- (2, 0)
_edj3zGm--_ (2, 1)
_edj3zGm--A (2, 2)
_edj3zGm--B (2, 3)
_edj3zGm--C (2, 4)
_edj3zGm--D (2, 5)
_edj3zGm--E (2, 6)
_edj3zGm--F (2, 7)
_edj3zGm--G (2, 8)
_edj3zGm--H (2, 9)
00jjnx (3, 0)
0072rt (3, 1)
005uzp (3, 2)
006kri (3, 3)
004oib (3, 4)
00hx5i (3, 5)
001nmc (3, 6)
0083sc (3, 7)
002y9o (3, 8)
00hx5i (3, 9)
SOUTHWORTH BILL (4, 0)
DOLE ROBERT F (4, 1)
KWONG SUN HON (4, 2)
TAYLOR JOSEPH (4, 3)
HUNT ROGER B (4, 4)
GRIECO WILLIAM (4, 5)
BORGHESANI PHILIP E (4, 6)
GIBLI

In [ ]:
def not_in_edges(key1, key2):
    node1 = corp_db["indiv_data_deduped"].fetchDocument(key1)
    edges = corp_db[]

In [21]:
print(indiv_ar.shape[0]/1000)

2613.278


In [36]:
indiv_ar[4][1][19:]

'12936271'

In [34]:
for i in range(indiv_ar.shape[0]):
    if c_1(indiv_ar[i],indiv_ar[4]):
        print(i)

4
278807
616535
822465
1050804
1151842


In [76]:
indiv_ar[0]

array(['12936266', 'indiv_data_deduped/12936266', '_edj3zGm---', '00jjnx',
       'SOUTHWORTH BILL', 'SOUTHWORTH BILL', ', ,',
       '112 N PILATUS LN, NAMPA, ID 83687'], dtype=object)

In [14]:
(', ,' in indiv_ar[:,6])

AttributeError: 'numpy.ndarray' object has no attribute 'find'

In [12]:
indiv_ar[282,6].find('AME, SAME')

1

In [51]:
print(len(indiv_ar))

2613278


In [99]:
indices = np.where((indiv_ar[:,6]!=indiv_ar[:,7]) & (indiv_ar[:,6]!=', ,') & (indiv_ar[:,7]!=', ,'))
print(indices)
print(len(indices[0]))

(array([    282,     288,     292, ..., 2612488, 2612503, 2612784],
      dtype=int64),)
19609


In [7]:
# extracting the keys of the indiv_data_processed collection
key_aql = "FOR x IN indiv_data_processed RETURN x._key"
key_query_result = corp_db.AQLQuery(key_aql,rawResults=True, batchSize = 1000)
indiv_keys = []

for key in key_query_result:
    indiv_keys.append(key)

In [8]:
indiv_1 = corp_db["indiv_data_processed"].fetchDocument(indiv_keys[0])
name_dict = {'IndividualName':'BURKE CHARLES'}
indivs = corp_db["indiv_data_processed"].fetchByExample(name_dict,batchSize = 1000)

BURKE CHARLES
